In [2]:
import torch
from torch.utils.data import Dataset, DataLoader
from pathlib import Path
import numpy as np
import cv2
import torchvision.transforms as transforms
from tqdm import tqdm
import torch.nn as nn
import torch.nn.functional as F

In [3]:
!jupyter nbextension enable --py widgetsnbextension

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: ok


In [4]:
class ThreatDataset(Dataset):
    def __init__(self, data, loader_type='train', transforms = None):
        self.folder_names = ['carrying', 'threat', 'normal']
        self.data = data
        self.transform = transforms
    
    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        data = self.data[idx]
        label = self.folder_names.index(data.parent.name)
        image = cv2.imread(str(data))
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        if self.transform:
            image = self.transform(image)
        return image, label

In [5]:
import random
from pathlib import Path

def split_data(data_dir, train_size=0.8, val_size = 0.1):
    random.seed(1234)
    data = Path('data').glob('*/*')
    data = [x for x in data if x.is_file() and x.suffix != '.zip']
    random.shuffle(data)
    train_size = int(len(data) * train_size)
    val_size = int(len(data) * val_size)
    train_data = data[:train_size]
    val_data = data[train_size:train_size+val_size]
    test_data = data[train_size+val_size:]

    return train_data, val_data, test_data


train_data, val_data, test_data = split_data('data')

In [6]:
train_transforms = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

val_transforms = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

test_transforms = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

transforms = {
    'train': train_transforms,
    'val': val_transforms,
    'test': test_transforms
}

train_dataset = ThreatDataset(train_data, loader_type="train", transforms=train_transforms)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

In [7]:
from scripts import dataloader

train_loader, val_loader, test_loader = dataloader.get_loaders()

In [14]:
x, y = iter(train_loader).next()
torch.argmax(F.one_hot(y, num_classes=3), 1)

tensor([1, 2, 2, 2, 2, 0, 0, 2, 1, 0, 0, 2, 1, 0, 2, 2, 0, 1, 1, 1, 2, 0, 1, 2,
        1, 1, 0, 0, 0, 2, 1, 2])

In [22]:
# For NLLLoss, we need to use the logit distribution
# https://discuss.pytorch.org/t/what-is-the-difference-between-nllloss-and-crossentropyloss/15553
# The CrossEntropyLoss combines the LogSoftmax and NLLLoss in one single class
# https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html
# https://pytorch.org/docs/stable/generated/torch.nn.NLLLoss.html

class Trainer:
    def __init__(self, model, optimizer, criterion, scheduler, device):
        # The trainer uses a one-hot distribution for the labels, so we need to use the CrossEntropyLoss
        # instead of the NLLLoss
        # Using FCC layer as the last layer, we can try to use basic loss functions like MSE or L1
        
        self.model = model
        self.optimizer = optimizer
        self.criterion = criterion
        self.scheduler = scheduler
        self.best_acc = 50
        if (device == 'cuda') and torch.cuda.is_available():
            self.device = torch.device('cuda')
        else:
            self.device = torch.device('cpu')

    def train(self, train_loader, val_loader, epochs=10):
        self.model.to(self.device)
        total = 0
        correct = 0
        for epoch in range(epochs):
            print(f"EPOCH {epoch}")
            self.model.train()
            tq = tqdm(enumerate(train_loader))
            for i, (x, y) in tq:
                x = x.to(self.device)
                y_label = y
                y = F.one_hot(y, num_classes=3).to(self.device).float()
                total += y.size(0)
                self.optimizer.zero_grad()
                y_pred = self.model(x)
                loss = self.criterion(y_pred, y)
                loss.backward()
                self.optimizer.step()
                
                # Calculate Accuracy - Only for softmax/logit distributions  
                _, predicted = torch.max(y_pred.data, 1)
                correct += (predicted == y_label).sum().item()
                tq.set_postfix(loss=loss.item(), acc=correct/total)
                if i % 100 == 0:
                    print(f'Epoch: {epoch}, Loss: {loss.item()}')
            self.validate(val_loader)
            print(f'Epoch: {epoch}, Accuracy: {correct/total}')

    def validate(self, val_loader):
        self.model.eval()
        total = 0
        correct = 0
        with torch.no_grad():
            tq = tqdm(enumerate(val_loader))
            for i, (x, y) in tq:
                x = x.to(self.device)
                y_label = y
                y = F.one_hot(y, num_classes=3).to(self.device).float()
                y_pred = self.model(x)
                loss = self.criterion(y_pred, y)

                total += y.size(0)
                _, predicted = torch.max(y_pred.data, 1)
                print(predicted)
                
                correct += (predicted == y_label).sum().item()
                if i % 100 == 0:
                    print(f'Validation Loss: {loss.item()}')
            print(f'Validation Accuracy: {correct/total}')
            if correct/total > self.best_acc:
                self.best_acc = correct/total
                print('Saving model...')
                torch.save(self.model.state_dict(), 'best_model.pth')
    
    def test(self, test_loader):
        self.model.eval()
        total = 0
        correct = 0
        with torch.no_grad():
            for i, (x, y) in tqdm(enumerate(test_loader)):
                x = x.to(self.device)
                y_label = y
                y = F.one_hot(y, num_classes=3).to(self.device).float()
                total += y.size(0)
                y_pred = self.model(x)
                loss = self.criterion(y_pred, y)

                _, predicted = torch.max(y_pred.data, 1)
                correct += (predicted == y_label).sum().item()
                if i % 100 == 0:
                    print(f'Test Loss: {loss.item()}')
        print(f'Accuracy: {100 * correct / total}')
    
    def save_model(self, path):
        torch.save(self.model.state_dict(), path)      

    def load_model(self, path):
        self.model.load_state_dict(torch.load(path)) 

In [23]:
from torchvision.models import resnet18
from torch.optim.lr_scheduler import StepLR
# from torch import nn
import torch.optim as optim 

model = resnet18(pretrained=True)
model.fc = nn.Linear(512, 3)

optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()
scheduler = StepLR(optimizer, step_size=5, gamma=0.1)

trainer = Trainer(model, optimizer, criterion, scheduler, 'cuda')
trainer.train(train_loader, val_loader, epochs=10)


EPOCH 0


1it [00:09,  9.16s/it, acc=0.406, loss=1.1]

Epoch: 0, Loss: 1.1015962362289429


9it [01:03,  7.02s/it, acc=0.365, loss=1.73]


KeyboardInterrupt: 